# Algorithm 16: Confidence Module (Boltz)

Predicts confidence scores (pLDDT, pAE, pTM).

## Source Code Location
- **File**: `Boltz-Ref-src/boltz-official/src/boltz/model/modules/confidence.py`
- **Class**: `ConfidenceModule`

In [ ]:
import numpy as np
np.random.seed(42)

def layer_norm(x, eps=1e-5):
    mean = np.mean(x, axis=-1, keepdims=True)
    var = np.var(x, axis=-1, keepdims=True)
    return (x - mean) / np.sqrt(var + eps)

def softmax(x, axis=-1):
    x_max = np.max(x, axis=axis, keepdims=True)
    exp_x = np.exp(x - x_max)
    return exp_x / np.sum(exp_x, axis=axis, keepdims=True)

In [ ]:
def plddt_head(s, num_bins=50):
    """
    Predicted LDDT head.
    
    Args:
        s: Single representation [N, c_s]
        num_bins: Number of pLDDT bins
    
    Returns:
        pLDDT logits and scores
    """
    N, c_s = s.shape
    
    s_norm = layer_norm(s)
    W = np.random.randn(c_s, num_bins) * (c_s ** -0.5)
    logits = s_norm @ W
    
    probs = softmax(logits)
    bin_centers = np.linspace(0, 1, num_bins) * 100
    plddt = np.sum(probs * bin_centers, axis=-1)
    
    return logits, plddt

In [ ]:
def pae_head(z, num_bins=64, max_error=32.0):
    """
    Predicted Aligned Error head.
    
    Args:
        z: Pair representation [N, N, c_z]
        num_bins: Number of error bins
        max_error: Maximum error in Angstroms
    
    Returns:
        PAE logits and expected error
    """
    N = z.shape[0]
    c_z = z.shape[-1]
    
    z_norm = layer_norm(z)
    W = np.random.randn(c_z, num_bins) * (c_z ** -0.5)
    logits = np.einsum('ijc,cb->ijb', z_norm, W)
    
    probs = softmax(logits)
    bin_centers = np.linspace(0, max_error, num_bins)
    pae = np.sum(probs * bin_centers, axis=-1)
    
    return logits, pae

In [ ]:
def pde_head(z, num_bins=64, max_error=32.0):
    """
    Predicted Distance Error head.
    
    Similar to PAE but for pairwise distance errors.
    """
    # Symmetrize
    z_sym = (z + z.transpose(1, 0, 2)) / 2
    return pae_head(z_sym, num_bins, max_error)

In [ ]:
def confidence_module(s, z):
    """
    Full Confidence Module.
    
    Args:
        s: Single representation [N, c_s]
        z: Pair representation [N, N, c_z]
    
    Returns:
        Dictionary of confidence scores
    """
    print(f"Confidence Module")
    print(f"="*50)
    
    # pLDDT
    plddt_logits, plddt = plddt_head(s)
    print(f"pLDDT: mean={plddt.mean():.1f}, range=[{plddt.min():.1f}, {plddt.max():.1f}]")
    
    # PAE
    pae_logits, pae = pae_head(z)
    print(f"PAE: mean={pae.mean():.2f}A")
    
    # PDE
    pde_logits, pde = pde_head(z)
    print(f"PDE: mean={pde.mean():.2f}A")
    
    return {
        'plddt': plddt,
        'pae': pae,
        'pde': pde,
    }

In [ ]:
# Test
print("Test: Confidence Module")
print("="*60)

N = 32
c_s = 128
c_z = 64

s = np.random.randn(N, c_s)
z = np.random.randn(N, N, c_z)

confidence = confidence_module(s, z)

print(f"\nShapes:")
for k, v in confidence.items():
    print(f"  {k}: {v.shape}")

## Key Insights

1. **pLDDT**: Per-residue confidence (0-100)
2. **PAE**: Predicted Aligned Error (asymmetric)
3. **PDE**: Predicted Distance Error (symmetric)
4. **Binned Output**: Classification over bins